In [3]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

import pandas as pd
import os

pd.options.mode.chained_assignment = None
import numpy as np
# Define the directory path where the files are located

# Initialize an empty DataFrame to store the merged data
df =  pd.read_excel("../../dataset/processed_data/main/timeseries/final_cleaned_TimeSeries_dataset.xlsx"  )

df.head()


,Unnamed: 0,District,"Laboratory confirmed, since the beginning of the pandemic TOTAL","Laboratory confirmed, since the beginning of the pandemic Difference from","Laboratory confirmed, since the beginning of the pandemic Hospitalized","Laboratory confirmed, since the beginning of the pandemic Deceased","Laboratory confirmed, since the beginning of the pandemic RecoveredA","Laboratory confirmed, since the beginning of the pandemic Current CasesB","Reported, the last 7 days,Incidence per 100, Incidence per 100 ,+USAFD","Reported, the last 7 days,Incidence per 100, Incidence per 100 ,<20 Jahre","Reported, the last 7 days,Incidence per 100, Incidence per 100 ,20-59 JahreE","Reported, the last 7 days,Incidence per 100, Incidence per 100 ,>=60 Jahre",Date
0,0,Ahrweiler,4577,4,199,52.0,4423,102,50.0,132.2,49.0,7.1,2021-08-01
1,1,Altenkirchen,4956,0,411,101.0,4833,22,8.5,8.4,12.1,2.6,2021-08-01
2,2,Alzey-Worms,5130,1,304,123.0,4970,37,9.3,24.0,4.4,8.2,2021-08-01
3,3,Bad Dürkheim,4499,1,305,150.0,4303,46,14.3,21.6,19.5,2.3,2021-08-01
4,4,Bad Kreuznach,6121,0,160,142.0,5923,56,18.3,41.4,20.0,2.0,2021-08-01


In [4]:
del df[df. columns[0]] 

In [5]:
# def convert_row_to_column_header(df):
#     new_header = df.iloc[0]  # Get the values from row 0
#     df = df[1:]  # Exclude row 0 from the DataFrame
#     df.columns = new_header  # Set the new column headers
    
#     return df

# df = convert_row_to_column_header(df)

In [6]:
def calculate_null_percentage(df):
    # Calculate the total number of rows in the DataFrame
    total_rows = df.shape[0]
    
    # Calculate the percentage of null values for each column
    null_percentage = ((  df.isnull().sum()  ) / total_rows) * 100
    
    # Print the columns with their corresponding null percentages
    print("Columns with null percentage:")
    print(null_percentage)



calculate_null_percentage(df)

Columns with null percentage:
District                                                                        0.0
Laboratory confirmed, since the beginning of the pandemic TOTAL                 0.0
Laboratory confirmed, since the beginning of the pandemic Difference from       0.0
Laboratory confirmed, since the beginning of the pandemic Hospitalized          0.0
Laboratory confirmed, since the beginning of the pandemic Deceased              0.0
Laboratory confirmed, since the beginning of the pandemic RecoveredA            0.0
Laboratory confirmed, since the beginning of the pandemic Current CasesB        0.0
Reported, the last 7 days,Incidence per 100, Incidence per 100 ,+USAFD          0.0
Reported, the last 7 days,Incidence per 100, Incidence per 100 ,<20 Jahre       0.0
Reported, the last 7 days,Incidence per 100, Incidence per 100 ,20-59 JahreE    0.0
Reported, the last 7 days,Incidence per 100, Incidence per 100 ,>=60 Jahre      0.0
Date                                          

In [ ]:
# Convert the 'date_column' to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Verify the column has been converted


In [ ]:
print(df.dtypes)

In [ ]:
df.head()

In [7]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense

# # Prepare the data
# # Assuming you have a time series dataset in CSV format with one input feature and one output/target variable
# df = pd.read_csv('your_data.csv')
# df['Date'] = pd.to_datetime(df['Date'])  # Convert 'Date' column to datetime format
df = df.set_index('Date')  # Set 'Date' column as the index

In [8]:
# print(df.shape[1])

In [23]:
# input_feature = df['input_feature'].values
# target_variable = df[].values
# target_variable = 'Laboratory confirmed, since the beginning of the pandemic Deceased'
target_variable = 'Laboratory confirmed, since the beginning of the pandemic Hospitalized'

#The next line should be replaced with imputation
# input_features = df.drop([""], axis=1).values


input_features = df.drop([target_variable,"District"], axis=1).values

# Extract target variable column
target_variable = df[target_variable].values


# Define the number of time steps and features
# You can adjust these values according to your dataset
time_steps = 10  # Number of time steps to consider for each sample
n_features = 9   # Number of input features

# Split the data into input sequences and corresponding outputs
X, y = [], []
for i in range(len(df) - time_steps):
    X.append(input_features[i:i+time_steps])
    y.append(target_variable[i+time_steps])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape the input data to match the LSTM input shape
X_train = np.reshape(X_train, (X_train.shape[0], time_steps, n_features))
X_test = np.reshape(X_test, (X_test.shape[0], time_steps, n_features))

X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, n_features)))
# model.add(LSTM(30, return_sequences=True))  # Added return_sequences=True
model.add(LSTM(20))  # Added another LSTM layer
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=10, batch_size=64)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test loss:", loss)

# # Make predictions
# # Assuming you have new data for prediction in the variable `new_df`
# new_df = pd.DataFrame({'Date': pd.date_range(start='2023-01-01', periods=10), 'input_feature': [1, 2, 3, 4, 5, 6, 7, 8, 9]})  # Example input data
# new_df['Date'] = pd.to_datetime(new_df['Date'])  # Convert 'Date' column to datetime format
# new_df = new_df.set_index('Date')  # Set 'Date' column as the index
# new_input = new_df['input_feature'].values
# new_input = np.reshape(new_input, (1, time_steps, n_features))
# prediction = model.predict(new_input)
# print("Prediction:", prediction)

Epoch 1/10
209/209 [==============================] - 5s 11ms/step - loss: 281794.5625
Epoch 2/10
209/209 [==============================] - 2s 9ms/step - loss: 276534.4375
Epoch 3/10
209/209 [==============================] - 2s 9ms/step - loss: 272414.5000
Epoch 4/10
209/209 [==============================] - 2s 9ms/step - loss: 268463.0625
Epoch 5/10
209/209 [==============================] - 2s 8ms/step - loss: 264616.5938
Epoch 6/10
209/209 [==============================] - 2s 9ms/step - loss: 260847.3438
Epoch 7/10
209/209 [==============================] - 2s 8ms/step - loss: 257144.6719
Epoch 8/10
209/209 [==============================] - 2s 8ms/step - loss: 253501.5781
Epoch 9/10
209/209 [==============================] - 2s 10ms/step - loss: 249911.1094
Epoch 10/10
105/105 [==============================] - 1s 7ms/step - loss: 952960.6875
Test loss: 952960.6875


In [30]:
model.fit(X_train, y_train, epochs=100, batch_size=64)

Epoch 1/100
209/209 [==============================] - 2s 11ms/step - loss: 83927.3203
Epoch 2/100
209/209 [==============================] - 2s 8ms/step - loss: 83307.0547
Epoch 3/100
209/209 [==============================] - 2s 9ms/step - loss: 82716.2344
Epoch 4/100
209/209 [==============================] - 2s 8ms/step - loss: 82154.0000
Epoch 5/100
209/209 [==============================] - 2s 8ms/step - loss: 81619.8750
Epoch 6/100
209/209 [==============================] - 2s 8ms/step - loss: 81114.6953
Epoch 7/100
209/209 [==============================] - 2s 8ms/step - loss: 80639.2734
Epoch 8/100
209/209 [==============================] - 2s 8ms/step - loss: 80190.6172
Epoch 9/100
209/209 [==============================] - 2s 9ms/step - loss: 79769.2344
Epoch 10/100
209/209 [==============================] - 2s 10ms/step - loss: 79375.2188
Epoch 11/100
209/209 [==============================] - 2s 9ms/step - loss: 79005.2344
Epoch 12/100
209/209 [============================

KeyboardInterrupt: 